# 矩阵分解模型

### tqdm库
- 进度条
- https://blog.csdn.net/qq_41731861/article/details/121678411

### np.bincount()
- https://blog.csdn.net/HG0724/article/details/121270521
- 统计从0到array数组中最大数字出现的个数的函数，并同样以array数组输出显示

### np.ones()
- 返回给定形状和数据类型的新数组，其中元素的值设置为1
- https://blog.csdn.net/silent1cat/article/details/119883549

### np.ceil()
- 计算大于等于该值的最小整数
- https://blog.csdn.net/devinzkx/article/details/124618434

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm  # 进度条

data = np.loadtxt('data/movielens_100k.csv', delimiter=',', dtype=int)
# 数据中的三列依次表示：用户编号，电影编号，评分
print('数据集大小：',len(data))

数据集大小： 100000


In [13]:
# 将数据全转化为序列0开始
data[:,:2] = data[:,:2] - 1

# 计算用户和电影数量
users = set()
items = set()
for i, j, k in data:
    users.add(i)
    items.add(j)
user_num = len(users)
item_num = len(items)
print(f'用户数{user_num},电影数{item_num}')

用户数943,电影数1682


In [15]:
# 划分训练集与测试集
np.random.seed(2023)
ratio = 0.8
split = int(len(data) * ratio)
np.random.shuffle(data)
train = data[:split]
test = data[split:]

# 统计训练集中每个用户和电影出现的次数，作为正则化约束的强度
user_cnt = np.bincount(train[:,0],minlength=user_num)
item_cnt = np.bincount(train[:,1],minlength=item_num)
print(user_cnt[:10],item_cnt[:10])

[228  51  47  17 148 156 336  41  16 149] [351 107  69 163  69  22 317 190 242  66]
